<a href="https://colab.research.google.com/github/MariaElisaOliveiraMartins/Curso_DS_Pro/blob/main/Aula8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 08
## AutoML 

## Definição 


“AutoML tenta automatizar a parte de ajuste de ML da ciência de dados”, o que significa que o AutoML tenta automatizar partes do fluxo de trabalho da ciência de dados.

Antes de começarmos, vamos entender as palavras-chave do aprendizado de máquina e da ciência de dados, o aprendizado de máquina trata principalmente principalmente da modelagem de dados (selecionar o melhor algoritmo, ajustar seus parâmetros etc.) é apenas uma parte do kit de ferramentas de ciência de dados que envolve ciência de dados, como preparação de dados, análise descritiva etc.

Então, basicamente, o autoML é um processo automático de seleção de algoritmo de modelo, ajuste de hiperparâmetro, modelagem iterativa e avaliação de modelo. NÃO é ciência de dados automatizada, nem desenvolvimento automatizado de inteligência artificial.

### Alternativas existentes

**auto-sklearn** é uma camada de otimização de hiperparâmetros bayesiana sobre o scikit-learn.

**TPOT** (TeaPOT) é uma biblioteca Python que cria e otimiza automaticamente pipelines completos de aprendizado de máquina usando programação genética.

**TransmogrifAI** é uma biblioteca Scala / SparkML criada pela Salesforce para limpeza automatizada de dados, engenharia de recursos, seleção de modelos e otimização de hiperparâmetros.

**H2O AutoML** realiza pré-processamento (simples) de dados, automatiza o processo de treinamento de grandes conjuntos de dados e modelos.

## H2O

O H2O.ai oferece uma variedade de plataformas de IA e de aprendizado de máquina, como a plataforma H2O de código aberto líder do setor, para integrações do Apache Spark Sparkling Water, a premiada plataforma de aprendizado automático de máquina, a plataforma AI Driverless H2O que aumenta e complementa os esforços de cientistas de dados. Por fim, o H2O Q é a nova plataforma de IA para usuários corporativos. 

### Instalando no Colab

In [ ]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

In [ ]:
! pip install h2o

### Instalando "local"

#### Instalar H2O no python:



1.   Requisitos

```
pip install requests
pip install tabulate
pip install "colorama>=0.3.8"
pip install future
```

2. H2O

```
pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
```

#### H2O Flow

1. Clique no botão **Download** que você verá em http://h2o-release.s3.amazonaws.com/h2o/latest_stable.html.

2. Crie um diretório, descompacte lá o arquivo baixado.

3. Execute:

```
java -jar h2o.jar
```
4. Aparecerá uma mensagem como:

```
Open H2O Flow in your web browser: http://10.212.60.3:54321
```
5. Abra esse link e aparecerá o H2O Flow






### Exercício 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import h2o

In [ ]:
h2o.init()

Dados: [exemplo sobre fraudes no Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud)

**Descrição:**

O conjunto de dados contêm transações realizadas com cartões de crédito em setembro de 2013 por portadores de cartões europeus.
Esse conjunto de dados apresenta transações que ocorreram em dois dias, nas quais temos 492 fraudes em 284.807 transações. O conjunto de dados é altamente desequilibrado, a classe positiva (fraudes) representa 0,172% de todas as transações.

Ele contém apenas variáveis ​​de entrada numéricas que são o resultado de uma transformação PCA. Infelizmente, devido a problemas de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Recursos V1, V2,… V28 são os principais componentes obtidos com o PCA, os únicos recursos que não foram transformados com o PCA são 'Time' e 'Value'. O recurso 'Hour' contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. O recurso 'Value' é o valor da transação, esse recurso pode ser usado para aprendizado sensível a custos dependente de exemplo. O recurso 'Class' é a variável de resposta e assume o valor 1 em caso de fraude e 0 em caso contrário.
	


In [ ]:
df = pd.read_csv('/content/drive/My Drive/DadosPythonPro/310_23498_bundle_archive.zip')

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.groupby('Class')['Class'].count()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(df[df.Class == 0].Time, df[df.Class == 0].Amount, c='green', label="Não fraudes")
plt.scatter(df[df.Class == 1].Time, df[df.Class == 1].Amount, c='red', label="Fraudes")
plt.title("Transações ao longo do tempo")
plt.legend()
plt.show()

In [ ]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.automl import H2OAutoML

In [ ]:
h2o.init()

In [ ]:
df = h2o.import_file('/content/drive/My Drive/DadosPythonPro/310_23498_bundle_archive.zip')

In [ ]:
d = df.split_frame(ratios = [0.8], seed = 1234)
df_train = d[0] # 80% para treinamento
df_test = d[1] # 20% para teste

In [ ]:
# Colunas de predição
predictors = list(df.columns) 
predictors.remove('Time')
predictors.remove('Class')

In [ ]:
# Convertendo para inteiros

df_train['Class'] = df_train['Class'].asfactor()

In [ ]:
aml = H2OAutoML(max_models = 3, seed = 1, max_runtime_secs=200)
aml.train(x = predictors, y = 'Class', training_frame = df_train, validation_frame = df_test)

In [ ]:
print(aml.leader.confusion_matrix)

In [ ]:
aml.leader

In [ ]:
# Salvando o modelo
h2o.save_model(aml.leader, path = "/content/drive/My Drive/credit_card")

### Exercicio 2

In [ ]:
# Obtendo os dados
df = h2o.import_file("https://github.com/h2oai/h2o-sparkling/raw/master/data/prostate.csv")


In [ ]:
df.head()

In [ ]:
# Checando o balanceamento
df.group_by("CAPSULE").count().get_frame()

In [ ]:
# Separando treinamento e teste
train, test = df.split_frame(ratios=[.9])
x = train.columns
y = "CAPSULE"
x.remove(y)

In [ ]:
# Convertendo para fator, já que é classificação
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
# "Rodando" os modelos por 60 segundos
aml = H2OAutoML(max_runtime_secs = 60)
aml.train(x = x, y = y, training_frame = train, leaderboard_frame = test)


In [ ]:
aml.leaderboard

In [ ]:
aml.leader

In [ ]:
# Fazendo predições
preds = aml.predict(test)
#preds = aml.leader.predict(test)

In [ ]:
preds